# Deep Dream 

`Deep Dream`은 2015년, 구글에서 발표된, CNN을 이용한 Image modification 기술이다. 아래의 그림과 같이 마치 꿈 속에서의 그림 처럼 보이고, 사진에서 벌레나, 개, 새 등의 사물이 추상적으로 표현된다는 특징 덕분에, 당시 인터넷에 크게 유행 하기도 했다. 

`Deep Dream`은 기존의 CNN의 필터를 시각화하는 작업과 맥을 같이 한다. `Deep Dream`은 아래와 같은 특징을 가지고 있다. 

  1. 전역적 경사 상승 법(Gradient Ascent)
    - CNN 필터를 시각화 하기 위해서, Convolutional Layer의 Output을 최대화 하는, Gradient Ascent 방법을 사용했다. `Deep Dream` 에서는 특정 레이어에만 이를 적용하는 것이 아니라, 다수의 레이어에 대해 적용하여, 많은 특징들을 한번에 표현하려 했다. 
  2. 실제 이미지를 입력
    - 빈 이미지나, 잡음이 심한 이미지로 부터 시각화 하는 것이 아니라, 실제 이미지로 부터 시각화 하기 때문에, 원래의 이미지의 시각적 패턴을 잘 가지게 되었다. 
  3. 여러 크기의 이미지를 처리 
    - 좀 더 나은 결과를 위해, 여러 크기의 이미지를 처리 하였다. 
 

In [5]:
# Load Pretrained model

from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing import image
import tensorflow as tf

K.set_learning_phase(0) 

model = inception_v3.InceptionV3(include_top=False, weights='imagenet')
# include_top : 마지막 Fully Connectted layer를 포함하여 불러올것인가를 설정하는 것이다.

In [3]:
# loss에 반영할 레이어 와 그에 해당하는 가중치 
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)

for layer_name in layer_contributions:
    
    # Add the L2 norm of the features of a layer to the loss.
    coeff = layer_contributions[layer_name]
    x = layer_dict[layer_name].output

    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(x), 'float32'))
    
    loss += coeff * K.sum(K.square(x[:, 2:-2, 2:-2, :])) / scaling

In [4]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]

fetch_loss_and_grads = K.Function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])  ## outputs 
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

#### -> 그냥 이렇게, loss_value, grad_values = fetch_loss_and_grads([dream])

def gradient_ascent(x, iterations, step, max_loss=None):

    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return sp.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    sp.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    
    return img

def deprocess_image(x):
    
    x = x.reshape((x.shape[1], x.shape[2], 3))
    
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype("uint8")
    return x

In [4]:
#### Parameters ####
K.get_session().run(tf.global_variables_initializer())
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 30
max_loss = 10.
fname = 'result_deepdream.png'

from skimage import data

# # img = data.chelsea()
# img = sp.misc.face()
# img = image.img_to_array(img)
# img = np.expand_dims(img, axis=0)
# img = inception_v3.preprocess_input(img)

img = preprocess_image("./deepdream_parc.png")

original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1] # 스케일 리스트 

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0]) # starting from smallest image

for shape in successive_shapes:
    
    print('Processing image shape', shape)
    
    img = resize_img(img, shape)
    
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)

    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)

    same_size_original = resize_img(original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    
    shrunk_original_img = resize_img(original_img, shape)
    
# plt.imshow(deprocess_image(img))
# image.save_img('result_deepdream.png', deprocess_image(img))
save_img(img, fname)

Processing image shape (395, 395)
...Loss value at 0 : 0.00011823373
...Loss value at 1 : 0.000121185716
...Loss value at 2 : 0.00012419512
...Loss value at 3 : 0.00012726638
...Loss value at 4 : 0.00013041019
...Loss value at 5 : 0.00013361704
...Loss value at 6 : 0.0001368889
...Loss value at 7 : 0.00014023352
...Loss value at 8 : 0.00014365124
...Loss value at 9 : 0.00014715707
...Loss value at 10 : 0.00015074346
...Loss value at 11 : 0.00015440988
...Loss value at 12 : 0.00015816561
...Loss value at 13 : 0.00016203013
...Loss value at 14 : 0.00016599047
...Loss value at 15 : 0.00017003718
...Loss value at 16 : 0.00017418477
...Loss value at 17 : 0.00017843887
...Loss value at 18 : 0.00018281411
...Loss value at 19 : 0.00018728059
...Loss value at 20 : 0.00019186373
...Loss value at 21 : 0.00019654006
...Loss value at 22 : 0.00020131815
...Loss value at 23 : 0.0002061883
...Loss value at 24 : 0.00021116347
...Loss value at 25 : 0.00021625064
...Loss value at 26 : 0.00022146491
...Lo